In [1]:
import json
import pprint
import shutil
from collections import defaultdict
from functools import partial
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm

from common.constants import ACTION_NAMES_IN_TEST, ALL_ACTION_NAMES, LAB_NAMES_IN_TEST
from config_utils import base_model_from_file
from gbdt.configs import GBDT_TrainConfig
from gbdt.rebalance_utils import DurationStats, get_labs_in_test_with_action

pd.set_option("display.max_colwidth", None)

In [2]:
from common.constants import DEFAULT_DURATION_CAP
from gbdt.helpers import get_any_train_config, get_train_config
from gbdt.train import try_fill_test_fold_for_model_cv
from postprocess.postprocess_utils import (
    full_fix_model_for_submission,
    is_full_fix_model_for_submission_needed,
    top_models_by_oof_per_action,
    top_models_by_oof_per_action_per_lab,
)
from postprocess.submission_utils import make_ensemble_submission, make_submission

def take_optim_lgmb(name) -> bool:
    if not name.startswith("lgbm"):
        return False
    config = get_any_train_config(name)
    if not config:
        return False
    if config.lightgbm_config.n_estimators <= 300: # and config.lightgbm_config.max_depth <= 11:
        return True
    return False

def take(name):
    # if name.startswith("catb"):
    #     return False
    if not name.startswith("lgbm"):
        return True
    config = get_any_train_config(name)
    if not config:
        return False
    return config.lightgbm_config.n_estimators <= 300 

models_dict = top_models_by_oof_per_action_per_lab(
    # take_filter=take_optim_lgmb
    # take_filter=lambda name: name.startswith("lgbm") and not take_optim_lgmb(name)
    # take_filter=lambda name: name.startswith("lgbm")
    # take_filter=lambda name: name.startswith("xgboost") and "new-code" in name
    # take_filter=lambda name: take_optim_lgmb(name) or name.startswith("xgboost")
    # take_filter=take,
    return_list=True
)
make_ensemble_submission(submission_name="submissions/e-app-per-action-23-nov", models_dict=models_dict, rewrite=True)
# make_submission(submission_name="submissions/all-best-21-nov-test", models_dict=models_dict, rewrite=True)

100%|██████████| 4496/4496 [00:01<00:00, 3968.94it/s]


[FOLDER DELETE] submissions/e-app-per-action-23-nov


Model lists per (action, lab):   0%|          | 0/89 [00:00<?, ?it/s]

Building ensemble: action=climb, lab=GroovyShrew, Approach: <=3, filter=0, ->nested_f1, bins=10, pool=1 ...


Model lists per (action, lab):   1%|          | 1/89 [00:06<09:25,  6.43s/it]

...

Model lists per (action, lab):  99%|█████████▉| 88/89 [29:32<00:05,  5.31s/it]

catb-exploreobject-base-NiftyGoldfinch * 0.691
lgbm-exploreobject-base * 0.086
catb-exploreobject-feats1-passive-0.7-300 * 0.222
nested_f1      : 0.11389 --> 0.12835
oof_f1         : 0.11784 --> 0.12192
f1_per_fold    : 0.12917 --> 0.13890
oof_pr_auc     : 0.04527 --> 0.04389
pr_auc_per_fold: 0.06044 --> 0.06599
---
Building ensemble: action=sniffbody, lab=CautiousGiraffe, Approach: <=3, filter=0, ->nested_f1, bins=10, pool=1 ...


Model lists per (action, lab): 100%|██████████| 89/89 [29:33<00:00, 19.93s/it]


lgbm-sniffbody-base * 0.444
catb-sniffbody-feats1-passive--1-300 * 0.222
catb-sniffbody-feats0-passive-0.8-300 * 0.333
nested_f1      : 0.39857 --> 0.41687
oof_f1         : 0.39385 --> 0.40621
f1_per_fold    : 0.41573 --> 0.42892
oof_pr_auc     : 0.30606 --> 0.33125
pr_auc_per_fold: 0.36319 --> 0.37792
---
nested_f1       : 0.51690
oof_f1          : 0.53243
f1_per_fold     : 0.53701
oof_pr_auc      : 0.50923
pr_auc_per_fold : 0.52726


Writing submission models: 100%|██████████| 204/204 [00:41<00:00,  4.95it/s]
